In [5]:
import cv2
import pandas as pd

# Load video and eye-tracking data
video_path = "pilot2/sangsu/02_ElevatorTest.mp4"
data_path = "pilot2/processed/eyetracker.pkl"
data = pd.read_pickle(data_path)
eye_data = data[data['Scene']==2].copy()
output_video = "pilot2/processed/output_with_gaze.mp4"

# Ensure you're working with a copy of the DataFrame slice

# Now, you can safely modify gaze_data
eye_data[['gazeL_X', 'gazeL_Y']] = eye_data[['gazeL_X', 'gazeL_Y']].interpolate()

# Check for any remaining NaN values after interpolation
eye_data[['gazeL_X', 'gazeL_Y']] = eye_data[['gazeL_X', 'gazeL_Y']].fillna(0.5)
eye_data['normalized_Unitytime'] = eye_data.groupby('Scene')['Unitytime'].transform(lambda x: x - x.min())

# Initialize video capture and writer
cap = cv2.VideoCapture(video_path)
fps = int(cap.get(cv2.CAP_PROP_FPS))  # Frames per second of the video
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Set target resolution for output video
target_width, target_height = 1920, 1080  # Example resolution

# Initialize VideoWriter with target resolution
out = cv2.VideoWriter(output_video, cv2.VideoWriter_fourcc(*'mp4v'), fps, (target_width, target_height))

# Scale factor for Vive Pro Eye resolution to target video resolution
vive_width, vive_height = 2880, 1600  # Original Vive resolution
scale_x = target_width / vive_width
scale_y = target_height / vive_height

# Convert normalized Unitytime to frame indices
eye_data['frame_idx'] = (eye_data['normalized_Unitytime'] * fps).astype(int)

# Iterate over each frame of the video
frame_idx = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Resize the frame to target resolution
    frame_resized = cv2.resize(frame, (target_width, target_height))

    # Find the corresponding eye-tracking data for the current video frame
    # Check for gaze data that matches the current frame index
    gaze_data = eye_data[eye_data['frame_idx'] == frame_idx]

    # Debug: Check if there is any matching gaze data
    if gaze_data.empty:
        print(f"No gaze data found for frame {frame_idx}")
    else:
        # Retrieve gaze coordinates
        x, y = gaze_data.iloc[-1][['gazeL_X', 'gazeL_Y']]

        # Scale gaze coordinates to match video resolution
        x = int(x * scale_x)
        y = int(y * scale_y)

        # Draw the gaze point on the frame
        if 0 <= x < target_width and 0 <= y < target_height:
            cv2.circle(frame_resized, (x, y), 5, (0, 255, 0), -1)

    # Write the frame to the output video
    out.write(frame_resized)
    frame_idx += 1

cap.release()
out.release()

In [7]:
gaze_data

,Scene,Unitytime,validL,validR,gazeoriginL_X,gazeoriginL_Y,gazeoriginL_Z,gazeoriginR_X,gazeoriginR_Y,gazeoriginR_Z,...,pupilLSensorPosL_X,pupilLSensorPosL_Y,pupilLSensorPosL_Z,pupilLSensorPosR_X,pupilLSensorPosR_Y,pupilLSensorPosR_Z,convergence_distance_mm,convergence_distance_validity,normalized_Unitytime,frame_idx
1970-01-01 00:01:46.634,2.0,124.058868,1.0,1.0,33.297462,7.837543,-39.451504,-28.222294,5.566531,-33.954803,...,0.457008,0.410544,4.344025e-44,0.476975,0.445716,4.203895e-45,0.0,0.0,53.868065,1616
1970-01-01 00:01:46.636,2.0,124.060837,1.0,1.0,33.297871,7.841812,-39.471569,-28.221577,5.566205,-33.958038,...,0.456981,0.410551,4.344025e-44,0.476973,0.445716,4.203895e-45,0.0,0.0,53.870033,1616
1970-01-01 00:01:46.638,2.0,124.062798,1.0,1.0,33.298275,7.846081,-39.491634,-28.220860,5.565877,-33.961277,...,0.456954,0.410558,4.344025e-44,0.476972,0.445716,4.203895e-45,0.0,0.0,53.871994,1616
1970-01-01 00:01:46.640,2.0,124.064766,1.0,1.0,33.298683,7.850350,-39.511703,-28.220142,5.565551,-33.964516,...,0.456927,0.410565,4.344025e-44,0.476970,0.445716,4.203895e-45,0.0,0.0,53.873962,1616
1970-01-01 00:01:46.642,2.0,124.066734,1.0,1.0,33.298859,7.848841,-39.504608,-28.219610,5.565361,-33.966175,...,0.456886,0.410587,4.344025e-44,0.476930,0.445716,4.203895e-45,0.0,0.0,53.875931,1616
1970-01-01 00:01:46.644,2.0,124.068710,1.0,1.0,33.298931,7.844572,-39.484543,-28.219170,5.565238,-33.967083,...,0.456837,0.410616,4.344025e-44,0.476870,0.445716,4.203895e-45,0.0,0.0,53.877907,1616
1970-01-01 00:01:46.646,2.0,124.070679,1.0,1.0,33.299000,7.840303,-39.464478,-28.218729,5.565114,-33.967991,...,0.456789,0.410646,4.344025e-44,0.476811,0.445716,4.203895e-45,0.0,0.0,53.879875,1616
1970-01-01 00:01:46.648,2.0,124.072647,1.0,1.0,33.299068,7.836034,-39.444408,-28.218288,5.564990,-33.968899,...,0.456740,0.410675,4.344025e-44,0.476752,0.445716,4.203895e-45,0.0,0.0,53.881844,1616
1970-01-01 00:01:46.650,2.0,124.074615,1.0,1.0,33.299141,7.831765,-39.424343,-28.217846,5.564866,-33.969807,...,0.456691,0.410704,4.344025e-44,0.476693,0.445716,4.203895e-45,0.0,0.0,53.883812,1616
1970-01-01 00:01:46.652,2.0,124.076584,1.0,1.0,33.299232,7.827493,-39.404274,-28.217566,5.565242,-33.971371,...,0.456634,0.410734,4.344025e-44,0.476621,0.445705,4.203895e-45,0.0,0.0,53.885780,1616
